# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import json
import os

# Import API key
from api_keys import g_key




### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#output file 
#save df to csv
#ctweather_df.to_csv("CityWeatherData.csv")

ctweatherdata_csv = "../WeatherPy/CityWeatherData.csv"
loadctweather_df = pd.read_csv(ctweatherdata_csv)

loadctweather_df.head()



,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Windspeed
0,0,quatre cocos,75,MU,1586649007,83,-20.21,57.76,75.99,5.82
1,1,butaritari,88,KI,1586648915,74,3.07,172.79,84.04,10.36
2,2,narsaq,100,GL,1586648870,39,60.92,-46.05,46.40,38.03
3,3,tuktoyaktuk,20,CA,1586649008,84,69.45,-133.04,6.80,11.41
4,5,clyde river,20,CA,1586649008,84,70.47,-68.59,5.00,10.29


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#configure gmaps
gmaps.configure(api_key=g_key)

#making humidity weight
loadctweather_df = loadctweather_df.dropna()
humidity = loadctweather_df["Humidity"].astype(float)

#store lat/ long as locations
latlong = loadctweather_df[["Lat", "Lng"]].astype(float)

#create heatmap 
fig = gmaps.figure()

hmap_layer = gmaps.heatmap_layer(latlong, weights=humidity, dissipating=False, max_intensity=max(loadctweather_df["Humidity"]),
                                point_radius =1)
fig.add_layer(hmap_layer)

fig



Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
#narrow down cities 
narrow_df = loadctweather_df
#max temp less than 90 but higher than 75
narrow_df = narrow_df.loc[(narrow_df["Max Temp"]<90) & (narrow_df["Max Temp"]>75)]
#cloudiness less than 30 
narrow_df = narrow_df.loc[(narrow_df["Cloudiness"] <30)]
#humidity less than 85 but higher than 30
narrow_df = narrow_df.loc[(narrow_df["Humidity"]<85) & (narrow_df["Humidity"]>30)]
#wind speed less than 15 mph
narrow_df = narrow_df.loc[(narrow_df["Windspeed"] <15)]

#drop null
narrow_df = narrow_df.dropna()

narrow_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Windspeed
18,23,hilo,1,US,1586648714,65,19.73,-155.09,82.40,12.75
27,33,aden,24,YE,1586648968,81,12.78,45.04,81.10,8.28
29,35,dingle,28,IE,1586649013,71,11.00,122.67,81.97,4.68
56,64,hobyo,4,SO,1586648905,75,5.35,48.53,81.97,11.10
60,68,vaini,20,TO,1586648934,65,-21.20,-175.20,82.40,5.82
64,73,banjar,20,ID,1586649079,83,-8.19,114.97,82.40,3.36
65,74,hithadhoo,5,MV,1586649006,67,-0.60,73.08,84.36,8.81
80,92,isangel,20,VU,1586648996,58,-19.55,169.27,84.20,5.48
89,102,ahuimanu,20,US,1586649083,58,21.44,-157.84,82.40,12.75
99,112,port hedland,5,AU,1586649085,57,-20.32,118.57,80.60,8.05


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = narrow_df

#params fx to update iteration

params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    #get lat, lng 
    latitude =row["Lat"]
    longitude =row["Lng"]
    
    #change location in iteration
    params["location"] = f"{latitude},{longitude}" 
    
    api_url = "http://maps.googleapis.com/maps/api/places/nearbysearch/json"
   
    #call url
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    call_url= requests.get(api_url,params=params)
    #convert to json
    response = call_url.json()
    print (json.dumps(response, indent=2, sort_keys=True))
          
    #extract results
    results = response['results']

    try:
        #print message
        print (f" The closest hotel is {results[0]['name']}.")
        #input data into DF
        hotel_df.loc[index, 'Hotel Name']=results[0]['name']
        #increase response count
    except (KeyError, IndexError):
        print("Hotel info not found---SKIP")

print('''----------------------------
         #Hotel Data Retrieval Complete
         #------------------------------''')
    
    

Retrieving Results for Index 18: hilo.


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display Map